# Домашнє завдання: Математичне формулювання логістичної регресії

В цьому домашньому завданні ми реалізуємо логістичну регресію на `numpy`.
Ці завдання допоможуть вам ґрунтовно засвоїти основні концепції логістичної регресії та реалізувати їх на практиці 🔥


Необхідні імпорти:

In [ ]:
import numpy as np

#### Завдання 1: Реалізація функції сигмоїди
1. З використанням `numpy` напишіть функцію `sigmoid(z)` для обчислення значення сигмоїди згідно з формулою:
   $$
   \sigma(z) = \frac{1}{1 + e^{-z}}
   $$
2. Використовуючи цю функцію, обчисліть значення сигмоїди для наступних даних: $ z = [-2, -1, 0, 1, 2] $. Виведіть результат обчислень.


In [ ]:
def sigmoid(z):
  return 1 / (1 + np.exp(-z))

In [ ]:
z = np.array([-2, -1, 0, 1, 2])
result = sigmoid(z)

print(f"Значення сигмоїди: {result}")

Значення сигмоїди: [0.11920292 0.26894142 0.5        0.73105858 0.88079708]




#### Завдання 2: Реалізація функції гіпотези для логістичної регресії
1. Напишіть функцію `hypothesis(theta, X)`, яка обчислює гіпотезу для логістичної регресії, використовуючи функцію сигмоїди. Формула гіпотези:
   $$
   h_\theta(x) = \sigma(\theta^T x) = \frac{1}{1 + e^{-\theta^T x}}
   $$
2. Використайте функцію `hypothesis` для обчислення значень гіпотези для наступних даних:
   
   $\theta = [0.5, -0.5]$
   
   $X = \begin{bmatrix} 1 & 2 \\ 1 & -1 \\ 1 & 0 \\ 1 & 1 \end{bmatrix}$

  Виведіть результат обчислень.


In [ ]:
def hypothesis_vec(theta, x):
    return sigmoid(np.dot(theta, x))

In [ ]:
def hypothesis(theta, X):
  res = []
  for x_i in X:
    res.append(hypothesis_vec(theta, x_i))
  return res

Також для нашого випадку функцію гіпотези    $$
   h_\theta(x) = \sigma(\theta^T x) = \frac{1}{1 + e^{-\theta^T x}}
   $$

можемо записати у матричному вигляді
$$
   h_\theta(X) = \sigma(X\theta) = \frac{1}{1 + e^{-X\theta}}
   $$

In [ ]:
def hypothesis_marix(theta, X):
  theta_t = np.expand_dims(theta, axis=1)
  z = np.dot(X, theta)
  return sigmoid(z)

In [ ]:
theta = np.array([0.5, -0.5])
X = np.array([
    [1, 2],
    [1, -1],
    [1, 0],
    [1, 1]
])

In [ ]:
hypothesis(theta, X)

[0.3775406687981454, 0.7310585786300049, 0.6224593312018546, 0.5]

In [ ]:
hypothesis_marix(theta, X)

array([0.37754067, 0.73105858, 0.62245933, 0.5       ])

**Висновок**:

Переписавши функцію гіпотези отримані, ті ж самі значення.

#### Завдання 3: Реалізація функції для підрахунку градієнтів фукнції втрат
1. Напишіть функцію `compute_gradient(theta, X, y)`, яка обчислює градієнти функції втрат для логістичної регресії. Формула для обчислення градієнта:
   $$
   \frac{\partial L(\theta)}{\partial \theta_j} = \frac{1}{m} \sum_{i=1}^{m} \left[ (h_\theta(x^{(i)}) - y^{(i)}) x_j^{(i)} \right]
   $$
2. Використайте функцію `compute_gradient` для обчислення градієнтів для наступних даних:

  $\theta = [0.5, -0.5]$

  $X = \begin{bmatrix} 1 & 2 \\ 1 & -1 \\ 1 & 0 \\ 1 & 1 \end{bmatrix}$

  $y = [1, 0, 1, 0]$

  Виведіть результат обчислень.

In [ ]:
theta = np.array([0.5, -0.5])
X = np.array([
    [1, 2],
    [1, -1],
    [1, 0],
    [1, 1]
])
y = np.array([1, 0, 1, 0])

In [ ]:
def compute_gradient(theta, X, y):
  m, n = X.shape
  grads = np.zeros(n)
  for j in range(n):
    grad = 0
    for i in range(m):
      grad += (hypothesis_vec(theta, X[i]) - y[i]) * X[i, j]
    grads[j] = grad
  return grads * 1/m

compute_gradient(theta, X, y)

array([ 0.05776464, -0.36899431])

Можемо перейти від формули градієнту

   $$
   \frac{\partial L(\theta)}{\partial \theta_j} = \frac{1}{m} \sum_{i=1}^{m} \left[ (h_\theta(x^{(i)}) - y^{(i)}) x_j^{(i)} \right]
   $$

для кожного окремого параметру до матричного вигляду:


$$
   G = \frac{1}{m} X^T (h_\theta(X) - y)
$$

In [ ]:
def compute_gradient_matrix(theta, X, y):
  m = X.shape[0]
  h = hypothesis_marix(theta, X)
  grad = (1 /m) * np.dot(X.T, h - y)
  return grad

compute_gradient_matrix(theta, X, y)

array([ 0.05776464, -0.36899431])

**Висновок**:

Результати пошуку градієнтів за допомогою обох функцій дають нам однакові результати.


#### Завдання 4: Реалізація повного батч градієнтного спуску

**Задача:**
1. Напишіть функцію `full_batch_gradient_descent(X, y, lr=0.1, epochs=100)`, яка реалізує алгоритм Full градієнтного спуску для логістичної регресії. Використовуйте такі формули:
   - Гіпотеза: $ h_\theta(x) = \sigma(\theta^T x) $
   - Оновлення параметрів: $ \theta_j := \theta_j - \alpha \frac{\partial L(\theta)}{\partial \theta_j} $
2. Використайте функцію `full_batch_gradient_descent` для обчислення параметрів моделі на наступних даних:

  $X = \begin{bmatrix} 1 & 2 \\ 1 & -1 \\ 1 & 0 \\ 1 & 1 \end{bmatrix}$

  $y = [1, 0, 1, 0]$

  Увага! Матриця $X$ вже має стовпець одиниць і передбачається, що це. - стовпець для intercept - параметра зсуву.

  Виведіть результат обчислень.


In [ ]:
X = np.array([
    [1, 2],
    [1, -1],
    [1, 0],
    [1, 1]
])
y = np.array([1, 0, 1, 0])

Для використання у методі, що реалізує повний батч градієнтний спуск можна використовувати як метод, що ітеративно знаходить градієнти, так і той що вирішує задачу у матричному вигляді.

In [ ]:
def full_batch_gradient_descent(X, y, lr=0.1, epochs=100):
  m, n = X.shape
  theta = np.zeros(n)

  for _ in range(epochs):
    theta -= lr * compute_gradient(theta, X, y)
  return theta

In [ ]:
theta = full_batch_gradient_descent(X, y)
theta

array([-0.2893693 ,  0.77655125])

#### Завдання 5. Обчислення точності моделі

1. Напишіть функцію `predict_proba(theta, X)`, яка використовує знайдені параметри $\theta$ для обчислення ймовірностей належності поточного прикладу з даних до класу $y=1$ на основі значень $\sigma(\theta^T x)$.

2. Напишіть функцію `predict(theta, X, threshold=0.5)`, яка обчислює клас з передбаченої імовірності належності екземпляра до класу 1 з порогом 0.5. Тобто якщо ймовірність менше 0.5, то передбачаємо клас 0, інакше клас 1.

3. Напишіть функцію `accuracy(y_true, y_pred)`, яка обчислює точність моделі, визначивши частку правильно передбачених класів.

  Формула метрики Accuracy:
  $$
  \text{Accuracy} = \frac{\sum_{i=1}^{m} I(\hat{{y}^{(i)}} = y^{(i)})}{m}
  $$

  де $\hat{{y}^{(i)}}$ - передбачене значення класу, $I$ - індикаторна функція (яка дорівнює 1, якщо умова виконується, і 0 - якщо ні), $m$ - кількість прикладів.

4. Обчисліть з використанням даних в завданні 4 $X$, $y$ та обчислених коефіцієнтах $\theta$ та виведіть на екран:
  - передбачені моделлю імовірності належності кожного з екземплярів в матриці `X` до класу 1
  - класи кожного екземпляра з матриці `X`
  - точність моделі.

##### 5.1


In [ ]:
def predict_proba(theta, X):
  proba = sigmoid(np.dot(X, theta))
  return proba

##### 5.2


In [ ]:
def predict(theta, X, threshold=0.5):
  proba = predict_proba(theta, X)  # Обчислюємо ймовірності
  return (proba >= threshold).astype(int)

##### 5.3


In [ ]:
def accuracy(y_true, y_pred):
  m = y_true.shape[0]
  return (sum(y_true == y_pred)/m)

##### 5.4


In [ ]:
proba = predict_proba(theta, X)
predictions = predict(theta, X)
acc = accuracy(y, predictions)

print(f"Реальні значення: {y}")
print("Ймовірності належності до класу y=1:", proba)
print("Передбачені класи з порогом 0.5:", predictions)
print(f"Model accuracy: {acc}")

Реальні значення: [1 0 1 0]
Ймовірності належності до класу y=1: [0.77966809 0.25617965 0.42815828 0.61944235]
Передбачені класи з порогом 0.5: [1 0 0 1]
Model accuracy: 0.5


**Висновок**:

Бачимо, що модель помилилася двічі.

Розраховані значення точності: 0.5